In [3]:
from scipy.io import loadmat
import h5py
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

In [4]:
# Initialize variables
batchsize = 500
l1_reg = 1e-8
l2_reg = 5e-7
noutputs = 919

# Load test file
y = loadmat('data/test.mat')
testLabelMatrix = y['testdata']
test_data_raw = y['testxdata']
test_data = test_data_raw.transpose(0, 2, 1)

# Define model
model = tf.keras.models.Sequential([
		tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True), input_shape=(1000,4)),
		tf.keras.layers.SeparableConv1D(750, (16), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg)),
		tf.keras.layers.Convolution1D(360, (8), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg)),
		tf.keras.layers.MaxPooling1D(4),
		tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
		tf.keras.layers.Dropout(0.2),
		tf.keras.layers.BatchNormalization(),
		tf.keras.layers.AveragePooling1D(8),
		tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
		tf.keras.layers.Flatten(),
		tf.keras.layers.Dense(noutputs, activation='sigmoid')
])

In [5]:
# Weight file to load in
weight_file = "ChromDL_best_weights"

# Load best model weights and calculate predictions
model.load_weights(weight_file + "/variables/variables")
model.compile(loss='binary_crossentropy', optimizer='ADAM', metrics=['accuracy'])
predProbs = model.predict(test_data, batch_size=batchsize, verbose=1)

# Save predictions to file
f = h5py.File("predProbs.hdf5", "w")
f.create_dataset("pred", data=predProbs)
f.close()

911/911 [==============================] - 468s 501ms/step


In [6]:
# Calculate auROC/auPRC metrics
aucs = np.zeros(noutputs, dtype=float)
auprcs = np.zeros(noutputs, dtype=float)
for i in range(noutputs):
    try:
        aucs[i] = roc_auc_score(testLabelMatrix[:, i], predProbs[:, i])
        auprcs[i] = average_precision_score(testLabelMatrix[:, i], predProbs[:, i])
    except ValueError:
        pass

histauc, histauprc = aucs[815:], auprcs[815:]
dnauc, dnauprc = aucs[:125], auprcs[:125]
tfauc, tfauprc = aucs[125:815], auprcs[125:815]

In [7]:
# Print results of testing
print(f"auROC list:\n{str(list(aucs))}")
print(f"auPRC list:\n{str(list(auprcs))}\n")

print(f"Transcription factors ({str(len(tfauc))}) samples:")
print(f" - Mean AUC: {str(np.nanmean(tfauc))}")
print(f" - Mean AUPRC: {str(np.nanmean(tfauprc))}")

print(f"DNase I-hypersensitive sites ({str(len(dnauc))}) samples:")
print(f" - Mean AUC: {str(np.nanmean(dnauc))}")
print(f" - Mean AUPRC: {str(np.nanmean(dnauprc))}")

print(f"Histone marks: ({str(len(histauc))}) samples:")
print(f" - Mean AUC: {str(np.nanmean(histauc))}")
print(f" - Mean AUPRC: {str(np.nanmean(histauprc))}")

print(f"Overall: ({str(len(tfauc) + len(dnauc) + len(histauc))}) samples:")
print(f" - Mean AUC: {str(np.nanmean(aucs))}")
print(f" - Mean AUPRC: {str(np.nanmean(auprcs))}")

auROC list:
[0.9148589115172916, 0.9369104275907696, 0.9028636465492857, 0.9339481106020286, 0.8444018809767091, 0.8950189623708099, 0.9218731237585517, 0.936877255015423, 0.930367947928027, 0.9384525753906399, 0.9280609795017898, 0.9350042778290322, 0.9080385034654206, 0.9525223226107591, 0.9226564891253153, 0.8848255005369966, 0.9445409436419743, 0.9421190744918457, 0.9374245582778818, 0.9135572757253756, 0.9291140883750406, 0.9517202443446741, 0.9340341232745304, 0.9332722237703754, 0.9337096212626983, 0.9161950033652941, 0.8699736955333095, 0.8553492054029199, 0.9263033042141441, 0.8567392145347501, 0.9258201726985024, 0.896393062968084, 0.9051504269636521, 0.9258748889752797, 0.9435887981223395, 0.9383756372481222, 0.9064223935869835, 0.8886288426501149, 0.9339465656182423, 0.9153351011307296, 0.9466674425190862, 0.9461366610681125, 0.9470972766994848, 0.9455306564442757, 0.9504732512660541, 0.9429975346498243, 0.9338242015813321, 0.9455908205968464, 0.9642891166821438, 0.92727218